In [1]:
import numpy as np
import pandas as pd
import ast



In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')


In [5]:
movies = movies.merge(credits, on='title')



In [7]:
movies = movies[['movie_id','title','overview','genres','cast','crew']]


In [9]:
movies.dropna(inplace=True)


In [11]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name'])
    return L


In [13]:
def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L

movies['crew'] = movies['crew'].apply(fetch_director)


In [15]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())


In [17]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [19]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['crew']


In [21]:
new_df = movies[['movie_id','title','tags']]


In [23]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


C:\Users\KIIT0001\AppData\Local\Temp\ipykernel_33304\1425802617.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))


In [25]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(new_df['tags']).toarray()


In [27]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vectors)


In [29]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(
        list(enumerate(distances)),
        reverse=True,
        key=lambda x:x[1]
    )[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)


In [31]:
recommend("Avatar")


Apollo 18
Beowulf
Tears of the Sun
The American
The Book of Life


In [33]:
import streamlit as st

st.title("🎬 Movie Recommendation System")

selected_movie = st.selectbox(
    "Select a movie",
    new_df['title'].values
)

if st.button("Recommend"):
    recommend(selected_movie)


2026-01-14 13:22:29.703 
  command:

    streamlit run C:\Users\KIIT0001\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [35]:
import pickle

pickle.dump(new_df, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

